In [2]:
import json

events = []
with open('out.txt') as fp_:
    for line in fp_:
        events.append(json.loads(line))

len(events)

29643

In [3]:
unique = set({e['inode'] for e in events})
len(unique)

111

In [4]:
events[0]

{'data': '00000000-0000-0000-0000-000000000000',
 'ts': 379283837619867,
 'pid': 2992215,
 'tgid': 2992210,
 'comm': 'glfs_sproc1',
 'function': 'getfattr',
 'inode': 658,
 'name': 'trusted.gfid',
 'ustack': ['lgetxattr',
  'posix_fill_gfid_path',
  'posix_pstat',
  'janitor_walker',
  'ftw_dir'],
 'kstack': ['zpl_xattr_get', 'kretprobe_trampoline']}

In [5]:
from uuid import UUID
nil = str(UUID(bytes=bytes([0]*16)))
nil

'00000000-0000-0000-0000-000000000000'

In [6]:
order = {}


for i, evt in enumerate(events):
    d = UUID(evt['data'])

    if evt['function'] == 'setfattr':
        exist = order.get(evt['inode'], None)

        if exist is not None:
            print(f'out of order set {evt["inode"]}: {d} != {exist}')
            continue

        order[evt['inode']] = evt['data']

out of order set 6526: 564fc8ef-4a33-4594-91cc-17542ae6c985 != a53c8d4b-b36a-44dd-b335-c538e4546596
out of order set 6394: f670ccbf-9a25-4412-8dfe-a1dfa4115d98 != f5f468ec-6bb7-450c-bbc0-ea32cb5cc7b0
out of order set 7450: ccd86c99-82a7-4273-89c4-547f155a5924 != c3f00577-f1b6-408f-b3fe-f85225003fec
out of order set 7450: 6b92f6c6-205a-43d3-a3ec-06a784a7a6ad != c3f00577-f1b6-408f-b3fe-f85225003fec
out of order set 8978: 03d809bd-252d-481f-8447-9da6f7f08911 != 9404e59b-09a8-4ff7-b7d4-da6aa191880c
out of order set 8002: 671d9ce0-3d6f-46ca-a97d-b8b96acfa624 != 3a5eaafc-38c3-40eb-9be9-9581eb45bfa5
out of order set 8002: 5a3d9e1d-e259-4282-8391-b86dd8e45c70 != 3a5eaafc-38c3-40eb-9be9-9581eb45bfa5


In [7]:
def print_event(evt):
    print(f'{evt["comm"]} {evt["function"]}({evt["inode"]}, {evt["name"]}) = {d}')


In [8]:
def print_stack(evt):
    trace = [*evt['kstack'], '-', *evt['ustack']]
    print('\n'.join(trace))

In [9]:
f = UUID('3a5eaafc-38c3-40eb-9be9-9581eb45bfa5')
ino = 8002

filtered = [
    evt for evt in events
    if UUID(evt['data']) == f
    or evt['inode'] == ino
]

for evt in filtered[:15]:
    d = UUID(evt['data'])
    print_event(evt)
    print_stack(evt)

glfs_iotwr00a getfattr(8002, trusted.gfid) = 00000000-0000-0000-0000-000000000000
zpl_xattr_get
kretprobe_trampoline
-
lgetxattr
posix_fill_gfid_path
posix_pstat
posix_lookup
default_lookup
default_lookup
br_stub_lookup
posix_acl_lookup
pl_lookup
default_lookup
default_lookup
default_lookup
up_lookup
default_lookup_resume
call_resume_wind
call_resume
iot_worker
start_thread
glfs_iotwr000 getfattr(8002, trusted.gfid) = 00000000-0000-0000-0000-000000000000
zpl_xattr_get
kretprobe_trampoline
-
lgetxattr
posix_fill_gfid_path
posix_pstat
posix_lookup
default_lookup
default_lookup
br_stub_lookup
posix_acl_lookup
pl_lookup
default_lookup
default_lookup
default_lookup
up_lookup
default_lookup_resume
call_resume_wind
call_resume
iot_worker
start_thread
glfs_iotwr00a getfattr(8002, trusted.gfid) = 00000000-0000-0000-0000-000000000000
zpl_xattr_get
kretprobe_trampoline
-
lgetxattr
posix_gfid_heal
posix_lookup
default_lookup
default_lookup
br_stub_lookup
posix_acl_lookup
pl_lookup
default_lookup
d